# Imports

In [ ]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score, accuracy_score, recall_score, roc_auc_score, ConfusionMatrixDisplay, RocCurveDisplay, precision_score
from lightgbm import LGBMRegressor as lgbm, early_stopping
import optuna
from boruta import BorutaPy

import requests
import zipfile
import io
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

from datetime import datetime
from meteostat import Point, Daily
from pathlib import Path
import time

# Step 1: Data Collection and Preprocessing

## Data Collection

### Import Data

In [ ]:
df = pd.read_csv("FINAL_DATASET.csv")

### Display Data

In [ ]:
print(df.head())
print("Data Range:")
print(df['FlightDate'].iloc[[0, -1]])

In [ ]:
df_grouped = df.groupby(['Origin', 'Dest', 'Month', 'DayOfWeek']).agg({
    'DepDelayMinutes': 'mean',
    'wspd': 'mean', 'tavg': 'mean', 'tmin': 'mean', 'tmax': 'mean', 'prcp': 'mean',
    'snow': 'mean', 'wdir': 'mean', 'wspd': 'mean', 'wpgt': 'mean', 'pres': 'mean',
}).reset_index()


## Data Preprocessing

### Converting to Imperial

In [ ]:
df = pd.read_csv("FINAL_DATASET.csv")

# Temperature: Convert °C to °F
for col in ["tavg", "tmin", "tmax"]:
    if col in df.columns:
        df[col] = (df[col] * 9/5 + 32).round().astype("Int64")

# Precipitation & Snow: Convert mm to inches
for col in ["prcp", "snow"]:
    if col in df.columns:
        df[col] = df[col] / 25.4

# Wind Speed: Convert km/h to mph
if "wspd" in df.columns:
    df["wspd"] = df["wspd"] / 1.60934

# Pressure: Convert hPa to inHg
if "pres" in df.columns:
    df["pres"] = df["pres"] * 0.02953

print(df.head(1))

### Converting Time to Cyclical

In [ ]:
# Transforming departure time to cyclical
print(df['CRSDepTime'])

df['DepHour'] = df['CRSDepTime'] // 100
df['DepMinute'] = df['CRSDepTime'] % 100
df['DepTotalMinutes'] = df['DepHour'] * 60 + df['DepMinute']

df['DepTime_sin'] = np.sin(2*np.pi*df['DepTotalMinutes'] / 1440)
df['DepTime_cos'] = np.cos(2*np.pi*df['DepTotalMinutes'] / 1440)

print(df['DepTime_sin'].head())
print(df['DepTime_cos'].head())

# Same for arrival
print(df['CRSArrTime'])

df['ArrHour'] = df['CRSArrTime'] // 100
df['ArrMinute'] = df['CRSArrTime'] % 100
df['ArrTotalMinutes'] = df['ArrHour'] * 60 + df['ArrMinute']

df['ArrTime_sin'] = np.sin(2*np.pi*df['ArrTotalMinutes'] / 1440)
df['ArrTime_cos'] = np.cos(2*np.pi*df['ArrTotalMinutes'] / 1440)

print(df['ArrTime_sin'].head())
print(df['ArrTime_cos'].head())

### Treating Missing Values

In [ ]:
# Percentage NA by column
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
percent_na = df.isna().mean() * 100
print(percent_na.sort_values(ascending = False).head(75))

# Removing >95% missing cols and IATA Code column due to redundancy
cols_to_drop = percent_na[percent_na > 95].index
df = df.drop(columns = cols_to_drop)
df = df.drop(columns = ["IATA_CODE_Reporting_Airline"])

print(f"\nDropped {len(cols_to_drop)} columns")
print("Remaining columns: ", len(df.columns))

# Printing percentage again
percent_na = df.isna().mean() * 100
print(percent_na.sort_values(ascending = False))

### Weather Graphs

In [ ]:
df['wspd'] = np.clip(df['wspd'], 0, 50) # Clip for realism
df['prcp'] = np.clip(df['prcp'], 0, 50) # Clip for realism
# Convert Snow_Presence to a descriptive category for better plotting
df['snow'] = df['snow'].map({0: 'No Snow', 1: 'Snow'})
# ----------------------------------------------------

# Set a style for better visualization
sns.set_style("whitegrid")

## 📊 Plotting Continuous Variables (Temperature, Wind Speed, Precipitation)

# Create a figure with subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15))
plt.suptitle('Distribution of Key Weather Factors', fontsize=16, y=1.02)

# --- 1. Temperature Distribution (Histogram with KDE) ---
sns.histplot(df['tavg'], bins=30, kde=True, color='skyblue', ax=axes[0])
axes[0].set_title('Temperature Distribution', fontsize=14)
axes[0].set_xlabel('Temperature (°F)')
axes[0].set_ylabel('Count / Density')
#

# --- 2. Wind Speed Distribution (Histogram with KDE) ---
sns.histplot(df['wspd'], bins=30, kde=True, color='orange', ax=axes[1])
axes[1].set_title('Wind Speed Distribution', fontsize=14)
axes[1].set_xlabel('Wind Speed (mph)')
axes[1].set_ylabel('Count / Density')
# Add a vertical line for the mean to highlight the average condition
axes[1].axvline(df['wspd'].mean(), color='r', linestyle='--', label=f"Mean: {df['wspd'].mean():.2f}")
axes[1].legend()

# --- 3. Precipitation Distribution (KDE Plot for Skewed Data) ---
# A KDE plot is often better for highly skewed data like precipitation,
# where many values are zero or near-zero.
sns.kdeplot(df['prcp'], fill=True, color='green', ax=axes[2],
            bw_adjust=0.5) # bw_adjust controls smoothness
axes[2].set_title('Precipitation Distribution', fontsize=14)
axes[2].set_xlabel('Precipitation (in)')
axes[2].set_ylabel('Density')


plt.tight_layout()
plt.show()

In [ ]:
weather_df = df[['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wspd', 'pres']]
plt.figure(figsize = (15, 10))
sns.heatmap(weather_df.corr(), annot = True, cmap = "coolwarm")
plt.show()

In [ ]:
weather_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wspd', 'pres']
n_cols = len(weather_cols)
fig, axes = plt.subplots(
    nrows=(n_cols + 1) // 2,
    ncols=2,
    figsize=(15, 5 * ((n_cols + 1) // 2))
)

axes = axes.flatten()

for i, col in enumerate(weather_cols):
    # 1. Create Bins (Categories) for the continuous variable

    # Use qcut to create 4 bins with roughly equal number of flights in each (quartiles)
    if col in ['tavg', 'tmin', 'tmax', 'wspd', 'pres']:
        df[f'{col}_Category'] = pd.qcut(df[col], q=4, labels=[f'Q1 ({col})', f'Q2 ({col})', f'Q3 ({col})', f'Q4 ({col})'])
        x_label = f'Quartile of {col} (e.g., Temperature, Wind Speed)'
    # Special handling for precipitation (prcp) and snow, as they often have many zeros
    elif col in ['prcp', 'snow']:
        # Create simple categories: Zero, Low, Medium, High
        if col == 'prcp':
            bins = [-np.inf, 0.01, 0.2, 1.0, np.inf]
            labels = ['None', 'Light', 'Moderate', 'Heavy']
        else: # snow
            bins = [-np.inf, 0.1, 1.0, 5.0, np.inf]
            labels = ['None/Trace', 'Light', 'Medium', 'Heavy']

        df[f'{col}_Category'] = pd.cut(df[col], bins=bins, labels=labels, right=True)
        x_label = f'{col} Category'

    # 2. Calculate the delay rate (mean of DepDel15) for each new category
    delay_summary = df.groupby(f'{col}_Category')['DepDel15'].mean().reset_index()
    delay_summary['Delay_Rate_Pct'] = delay_summary['DepDel15'] * 100

    # 3. Plot the result
    sns.barplot(
        x=f'{col}_Category',
        y='Delay_Rate_Pct',
        data=delay_summary,
        ax=axes[i],
        palette='coolwarm',
        edgecolor='black'
    )

    # Add values on top of the bars
    for index, row in delay_summary.iterrows():
        axes[i].text(row.name, row['Delay_Rate_Pct'] + 0.5, f"{row['Delay_Rate_Pct']:.1f}%",
                        color='black', ha="center", fontsize=8)

    axes[i].set_title(f'Delay Rate vs. {col.upper()}', fontsize=12)
    axes[i].set_xlabel(x_label, fontsize=10)
    axes[i].set_ylabel('Delay Rate (%)', fontsize=10)
    axes[i].set_ylim(0, delay_summary['Delay_Rate_Pct'].max() + 5)
    axes[i].tick_params(axis='x', rotation=15)
    axes[i].grid(axis='y', linestyle=':', alpha=0.6)

# Hide any unused subplots
for i in range(n_cols, len(axes)):
    fig.delaxes(axes[i])

fig.suptitle('Impact of Individual Weather Factors', fontsize=16, fontweight='bold', y=1.02)
plt.subplots_adjust(bottom=1, right=0.8, top=1)
plt.show()

In [ ]:
delay_col='DepDel15'
weather_cols=['tavg', 'prcp']
# Calculate the number of subplots needed
n_cols = len(weather_cols)
# Determine the number of rows based on the number of factors (2 plots per row)
nrows = (n_cols + 1) // 2
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=2,
    figsize=(15, 5 * nrows)
)

axes = axes.flatten()

for i, col in enumerate(weather_cols):

    # 1. Create Bins (Categories) for the continuous variable based on the factor

    if col in ['tavg', 'tmin', 'tmax']:
        # Custom bins for Temperature to capture cold and hot extremes (bimodal impact)
        bins = [-np.inf, 30, 45, 75, 90, np.inf]
        labels = ['Very Cold (<30°F)', 'Cold (30-45°F)', 'Moderate (45-75°F)', 'Hot (75-90°F)', 'Very Hot (>90°F)']
        # x_label = f'{col.upper()} Range (°F)'
        x_label = ''
        df[f'{col}_Category'] = pd.cut(df[col], bins=bins, labels=labels, right=True)

    elif col == 'wspd':
        # Custom bins for Wind Speed to capture critical thresholds (nonlinear impact)
        bins = [-np.inf, 10, 20, 30, np.inf]
        labels = ['Low (<10 mph)', 'Medium (10-20 mph)', 'High (20-30 mph)', 'Severe (>30 mph)']
        # x_label = 'Wind Speed Range (MPH)'
        df[f'{col}_Category'] = pd.cut(df[col], bins=bins, labels=labels, right=True)


    # 2. Calculate the delay rate (mean of DepDel15) for each new category
    category_col = f'{col}_Category'
    delay_summary = df.groupby(category_col)[delay_col].mean().reset_index()
    delay_summary['Delay_Rate_Pct'] = delay_summary[delay_col] * 100

    # 3. Plot the result
    sns.barplot(
        x=category_col,
        y='Delay_Rate_Pct',
        data=delay_summary,
        ax=axes[i],
        palette='coolwarm',
        edgecolor='black'
    )

    # Add values on top of the bars
    for index, row in delay_summary.iterrows():
        axes[i].text(row.name, row['Delay_Rate_Pct'] + 0.5, f"{row['Delay_Rate_Pct']:.1f}%",
                        color='black', ha="center", fontsize=8)

    axes[i].set_title(f'Delay Rate vs. {col.upper()}', fontsize=12)
    axes[i].set_xlabel(x_label, fontsize=10)
    axes[i].set_ylabel('Delay Rate (%)', fontsize=10)
    axes[i].set_ylim(0, delay_summary['Delay_Rate_Pct'].max() + 5)
    axes[i].tick_params(axis='x', rotation=15)
    axes[i].grid(axis='y', linestyle=':', alpha=0.6)

# Hide any unused subplots
for i in range(n_cols, len(axes)):
    fig.delaxes(axes[i])

fig.suptitle('Impact of Individual Weather Factors', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.98])
plt.show()

In [ ]:
df_delayed = df
df['TotalDisruptionMinutes'] = np.where(
    (df['Cancelled'] == 1.0) | (df['Diverted'] == 1.0),
    1000,
    df['DepDelayMinutes']
)

In [ ]:
# 1.1 IsSevereDisruption (Total disruption flag, used for reference)
df['IsSevereDisruption'] = np.where(
    (df['Cancelled'] == 1.0) | (df['Diverted'] == 1.0) | (df['DepDel15'] == 1.0),
    1.0,
    0.0
)

# 1.2 IsCancelledOrDiverted (The worst outcomes)
df['IsCancelledOrDiverted'] = np.where(
    (df['Cancelled'] == 1.0) | (df['Diverted'] == 1.0),
    1.0,
    0.0
)

# 1.3 IsDelayedOnly (15+ minute delays that were NOT Cancelled/Diverted)
df['IsDelayedOnly'] = np.where(
    (df['DepDel15'] == 1.0) & (df['IsCancelledOrDiverted'] == 0.0),
    1.0,
    0.0
)

In [ ]:
weather_cols = ['wspd']
n_cols = len(weather_cols)

# Set up the plot figure for a single bar chart
fig, axes = plt.subplots(
    nrows=1,
    ncols=1,
    figsize=(10, 7) # Increased size for clarity
)
axes = [axes]

for i, col in enumerate(weather_cols):

    # 1. Create Bins (Categories) for Wind Speed
    if col == 'wspd':
        bins = [-np.inf, 10, 20, 30, np.inf]
        labels = ['Low (<10 mph)', 'Medium (10-20 mph)', 'High (20-30 mph)', 'Severe (>30 mph)']
        x_label = 'Wind Speed Range (MPH)'
        df[f'{col}_Category'] = pd.cut(df[col], bins=bins, labels=labels, right=True)

    # 2. Calculate the rate for each mutually exclusive category
    category_col = f'{col}_Category'
    delay_summary = df.groupby(category_col).agg(
        Cancelled_Diverted_Rate=('IsCancelledOrDiverted', 'mean'),
        Delayed_Only_Rate=('IsDelayedOnly', 'mean'),
        Total_Disruption_Rate=('IsSevereDisruption', 'mean') # Calculate total for annotations
    ).reset_index()

    # 3. Reshape the data for a stacked bar plot (melt)
    summary_melted = delay_summary.melt(
        id_vars=[category_col, 'Total_Disruption_Rate'],
        value_vars=['Cancelled_Diverted_Rate', 'Delayed_Only_Rate'],
        var_name='Disruption_Type',
        value_name='Rate'
    )
    summary_melted['Rate_Pct'] = summary_melted['Rate'] * 100

    # Rename types for legend clarity
    summary_melted['Disruption_Type'] = summary_melted['Disruption_Type'].replace({
        'Cancelled_Diverted_Rate': 'Cancellation / Diversion',
        'Delayed_Only_Rate': '15+ Minute Delay Only'
    })

    # 4. Plot the stacked result
    sns.barplot(
        x=category_col,
        y='Rate_Pct',
        hue='Disruption_Type', # Use hue to create the segments
        data=summary_melted,
        ax=axes[i],
        palette='Set2', # A clearer palette for two groups
        edgecolor='black',
        dodge=False # CRITICAL: Set to False to make the bars stack
    )

    axes[i].set_title(f'Composition of Flight Disruption by Wind Speed', fontsize=14)
    axes[i].set_xlabel(x_label, fontsize=12)
    axes[i].set_ylabel('Disruption Rate (%)', fontsize=12)
    axes[i].set_ylim(0, delay_summary['Total_Disruption_Rate'].max() * 100 + 5)
    axes[i].tick_params(axis='x', rotation=15)
    axes[i].legend(title='Disruption Type')
    axes[i].grid(axis='y', linestyle=':', alpha=0.6)

fig.suptitle('Breakdown of Severe Disruption Risk (Delay vs. Cancellation/Diversion)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.98])
plt.show()

In [ ]:
delay_magnitude_col = 'TotalDisruptionMinutes'

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# --- Option A: Scatter Plot (Delay Magnitude vs. Wind Speed) ---
sns.scatterplot(
    x='wspd',
    y=delay_magnitude_col,
    data=df_delayed,
    alpha=0.4,
    ax=ax1,
    color='darkblue'
)
# Add a linear regression line for context
sns.regplot(
    x='wspd',
    y=delay_magnitude_col,
    data=df_delayed,
    scatter=False,
    color='red',
    line_kws={'linestyle': '--', 'label': 'Linear Trend'},
    ax=ax1
)

ax1.set_title('Scatter Plot: Wind Speed (wspd) vs. Delay Magnitude', fontsize=14)
ax1.set_xlabel('Wind Speed (Knots/MPH)', fontsize=12)
ax1.set_ylabel('Delay Magnitude (Minutes)', fontsize=12)
ax1.grid(axis='both', linestyle=':', alpha=0.6)

# --- Option B: Box Plot (Delay Magnitude by Wind Speed Bin) ---
wind_bins = [-np.inf, 10, 20, 30, np.inf]
wind_labels = ['<10 MPH', '10-20 MPH', '20-30 MPH', '>30 MPH']

# Use the same logic as in Slide 4 to create ordered bins
df_delayed['Wind_Bin_Label'] = pd.cut(
    df_delayed['wspd'],
    bins=wind_bins,
    labels=wind_labels,
    right=True,
    duplicates='drop'
)

sns.boxplot(
    x='Wind_Bin_Label', # Use the fixed bins
    y=delay_magnitude_col,
    data=df_delayed,
    ax=ax2,
    palette='plasma',
    notch=True
)

ax2.set_title('Box Plot: Delay Magnitude by Wind Speed Range', fontsize=14)
ax2.set_xlabel('Wind Speed Range', fontsize=12)
ax2.set_ylabel('Delay Magnitude (Minutes)', fontsize=12)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=15, ha='right')
ax2.grid(axis='y', linestyle=':', alpha=0.6)

fig.suptitle('Correlation Between Continuous Weather Factors and Delay Magnitude', fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.95]) # Adjust layout for suptitle
plt.show()

### Heatmap After Removing Highly Correlated and Unwanted Variables

In [ ]:
numeric_df = df.select_dtypes(include = ['number']).drop(columns = ['DepTime', 'Year', 'Quarter', 'DayofMonth', 'DivAirportLandings', 'DestAirportSeqID', 'Flights', 'OriginAirportID', 'OriginStateFips', 'DestWac', 'OriginWac', 'CRSDepTime', 'CRSArrTime', 'Cancelled', 'DestStateFips', 'DayOfWeek', 'Month', 'DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'OriginCityMarketID', 'Diverted', 'OriginAirportSeqID', 'DestAirportID', 'DestCityMarketID'], errors = 'ignore')
plt.figure(figsize = (30, 12))
sns.heatmap(numeric_df.corr(method='spearman'), annot = True, cmap = "coolwarm")
plt.show()

In [ ]:
sns.histplot(df['DepDelayMinutes'], bins=100)
plt.xlim(0, 300)
plt.show()

The extreme right skew, combined with the low correlation coefficients, indicates we need to perform some feature engineering to help the model find relations between weather data and delay lengths.

In [ ]:
percent_na = df.isna().mean() * 100
print(percent_na.sort_values(ascending = False))

### Delays Over Time

In [ ]:
# Ensure date column is datetime
df['FlightDate'] = pd.to_datetime(df['FlightDate'])

# ------------------------
# 2️⃣ Aggregate: average delay per day per airport
# ------------------------
avg_delay = (
    df.groupby(['FlightDate', 'Origin'])['DepDelayMinutes']
      .mean()
      .reset_index()
)

# ------------------------
# 3️⃣ Get unique airports
# ------------------------
airports = avg_delay['Origin'].unique()
num_airports = len(airports)

# ------------------------
# 4️⃣ Create subplots (one per airport)
# ------------------------
# Calculate number of rows and columns for subplots
cols = 2  # Adjust if you want more columns
rows = (num_airports + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(14, 4*rows), sharex=True, sharey=True)
axes = axes.flatten()  # Flatten to easily index

# Plot each airport
for i, airport in enumerate(airports):
    airport_data = avg_delay[avg_delay['Origin'] == airport]
    sns.lineplot(
        data=airport_data,
        x='FlightDate',
        y='DepDelayMinutes',
        ax=axes[i],
        color='blue'
    )
    axes[i].set_title(f"Airport: {airport}")
    axes[i].set_xlabel("Date")
    axes[i].set_ylabel("Avg Departure Delay (mins)")
    axes[i].grid(True)

# Remove unused subplots if any
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle("Average Departure Delay Over Time by Airport", y=1.02, fontsize=16)
plt.show()

### Transform Target

#### Log Transformation

In [ ]:
df['DepDelayMinutes_log'] = np.log1p(df['DepDelayMinutes'])

#### Box-Cox Transformation

In [ ]:
df['DepDelay_shifted'] = df['DepDelayMinutes'] + 1e-3  # avoid zeros
pt_boxcox = PowerTransformer(method='box-cox', standardize=False)
df['DepDelayMinutes_boxcox'] = pt_boxcox.fit_transform(df['DepDelay_shifted'].values.reshape(-1,1))

#### Yeo-Johnson Transformation

In [ ]:
pt_yeojohnson = PowerTransformer(method='yeo-johnson', standardize=False)
df['DepDelayMinutes_yeojohnson'] = pt_yeojohnson.fit_transform(df['DepDelayMinutes'].values.reshape(-1,1))

## Feature Engineering

### Extra Features

In [ ]:
df['FlightDate'] = pd.to_datetime(df['FlightDate'])

# DayOfYear: Day of the year (1-365/366)
df['DayOfYear'] = df['FlightDate'].dt.dayofyear

# IsWeekend: 1 if the flight date is Saturday or Sunday, else 0
df['IsWeekend'] = df['DayOfWeek'].isin([6,7]).astype(int)

# IsHoliday: 1 if the flight date is a US federal holiday, else 0
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df['FlightDate'].min(), end=df['FlightDate'].max())
df['IsHoliday'] = df['FlightDate'].isin(holidays)

# IsHolidayWindow: within 1 day before or after a holiday
holiday_window = pd.concat([
    pd.Series(holidays - pd.Timedelta(days=1)),
    pd.Series(holidays),
    pd.Series(holidays + pd.Timedelta(days=1))
])
df['IsHolidayWindow'] = df['FlightDate'].isin(holiday_window)

# NumDepartures: number of departures from the origin airport on that day
daily_departures = (
    df.groupby(['OriginAirportID', 'FlightDate'])
      .size()
      .rename('NumDepartures')
      .reset_index()
)

df = df.merge(daily_departures, on=['OriginAirportID', 'FlightDate'], how='left')

# Ensure Date column exists and is datetime
df['Date'] = pd.to_datetime(df['FlightDate'])

# Sort so rolling ops are time-consistent
df = df.sort_values(['OriginAirportID', 'Date'])

# RouteDelayMean_7d: Route-level rolling delay mean
df['Route'] = df['OriginAirportID'].astype(str) + '_' + df['DestAirportID'].astype(str)
df['RouteDelayMean_7d'] = (
    df.groupby('Route')['DepDelayMinutes']
      .transform(lambda x: x.shift().rolling(7, min_periods=1).mean())
)

# OriginDelayMean_7d: Airport-level (origin) rolling delay mean
df['OriginDelayMean_7d'] = (
    df.groupby('OriginAirportID')['DepDelayMinutes']
      .transform(lambda x: x.shift().rolling(7, min_periods=1).mean())
)

# DestArrivals_7d: Destination congestion indicator
df['DestArrivals_7d'] = (
    df.groupby('DestAirportID')['Flight_Number_Reporting_Airline']
      .transform(lambda x: x.shift().rolling(7, min_periods=1).count())
)

# Replace any remaining NaN (from shift/rolling) with reasonable defaults
df.fillna({
    'RouteDelayMean_7d': 0,
    'OriginDelayMean_7d': 0,
    'DestArrivals_7d': 0
}, inplace=True)

# Departures_Today: Day-level departure volume (can help with congestion)
df['Departures_Today'] = df.groupby(['OriginAirportID', 'Date'])['Flight_Number_Reporting_Airline'].transform('count')

# Interaction features (useful for nonlinear models like LightGBM)
df['Dist_x_Wspd'] = df['Distance'] * df['wspd']
df['TempRange'] = df['tmax'] - df['tmin']
df['MonthxWeekday'] = df['Month'] * df['DayOfWeek']

# CongestionRatio: Ratio of today's departures to average departures for that airport
df['AvgDepartures_Past30d'] = (
    df.groupby('OriginAirportID')['NumDepartures']
      .transform(lambda x: x.shift().rolling(30, min_periods=1).mean())
)
df['CongestionRatio'] = df['NumDepartures'] / df['AvgDepartures_Past30d']

# OriginDelayTrend_3d: Captures worsening congestion trend
df['OriginDelayTrend_3d'] = df.groupby('OriginAirportID')['OriginDelayMean_7d'].transform(lambda x: x.diff(3))



# Step 2: Modeling

## LightGBM

### Feature Selection & Splitting

In [ ]:
features = [

    # 'Year', Removing Year to focus on dynamic predictors
    'Month', 'DayOfWeek', 'Flight_Number_Reporting_Airline',
    'OriginAirportID', 'OriginCityMarketID', 'OriginStateFips',
    'DestAirportID', 'DestCityMarketID', 'DestStateFips',
    'CRSDepTime', 'CRSArrTime', 'Distance', 'tavg', 'tmin', 'tmax',
    'prcp', 'wspd', 'pres', 'IsWeekend', 'IsHoliday', 'IsHolidayWindow',
    'NumDepartures', 'RouteDelayMean_7d', 'OriginDelayMean_7d',
    'DestArrivals_7d', 'Departures_Today', 'Dist_x_Wspd', 'TempRange',
    'MonthxWeekday',
]
# One-hot encoded categorical variables
one_hot_cols = [col for col in df.columns if col.startswith('Origin_') or
                col.startswith('Month_') or
                col.startswith('DayOfWeek_') or
                col.startswith('Dest_')]

target = 'DepDelayMinutes_log'

# Define X and y
X = df[features + one_hot_cols]
y = df[target]

# Keep FlightDate in a separate series for splitting
flight_dates = df['FlightDate']

# Subsample for speed
df_model = df[features + one_hot_cols + [target]].dropna().sample(500_000, random_state=42)
flight_dates_model = flight_dates.loc[df_model.index]

# Time-based split
train_end = '2023-12-31'
valid_end = '2024-12-31'

train_mask = flight_dates_model <= train_end
valid_mask = (flight_dates_model > train_end) & (flight_dates_model <= valid_end)
test_mask = flight_dates_model > valid_end

# Define X and y
X = df_model[features + one_hot_cols]  # do NOT include FlightDate
y = df_model[target]

# Split and convert numeric to float32
X_train = X[train_mask].astype(np.float32)
X_valid = X[valid_mask].astype(np.float32)
X_test  = X[test_mask].astype(np.float32)

y_train = y[train_mask]
y_valid = y[valid_mask]
y_test  = y[test_mask]

### Hyperparameter Tuning

In [ ]:
def objective(trial):
    params = {
        'n_estimators': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 500),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 1.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 1.0),
        'random_state': 42,
        'n_jobs': -1
    }

    model = lgbm(**params)
    model.fit(
        X_train, y_train,  # y_train is log-transformed
        eval_set=[(X_valid, y_valid)],
        eval_metric='mae',
        callbacks=[early_stopping(stopping_rounds=100)]
    )

    # Back-transform predictions to minutes
    y_pred_log = model.predict(X_valid)
    y_pred = np.expm1(y_pred_log)
    y_valid_orig = np.expm1(y_valid)

    mae = mean_absolute_error(y_valid_orig, y_pred)
    return mae  # Optuna minimizes MAE in minutes

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)  # adjust trials as needed

print("Best hyperparameters:")
print(study.best_params)
print("Best MAE:", study.best_value)

### Forecasting

In [ ]:
# =========================
# Train LightGBM
# =========================
best_params = study.best_params
lgb = lgbm(**best_params)

lgb.fit(X_train, y_train)

# =========================
# Predict, back-transform, and evaluate
# =========================

y_pred_log = lgb.predict(X_test)
y_pred = np.expm1(y_pred_log)       # back to minutes
y_test_original = np.expm1(y_test)  # back to minutes

# Evaluate in log space
mae_log = mean_absolute_error(y_test, y_pred_log)
rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_log))
r2_log = r2_score(y_test, y_pred_log)

# Evaluate in original minutes
mae = mean_absolute_error(y_test_original, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred))
r2 = r2_score(y_test_original, y_pred)

print("\nModel Performance (LightGBM):")
print("\nIn Log Space:")
print(f"R²:  {r2_log:.4f}")
print("\nIn Original Minutes:")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")

# =========================
# Feature importance plot
# =========================
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': lgb.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x='Importance', y='Feature', data=importances.head(15), palette='viridis', legend=False)
plt.title("Top 15 Most Important Features (LightGBM)")
plt.tight_layout()
plt.show()

## Random Forest

### Pipeline

In [ ]:
# --- New, Simplified Training Pipeline ---
print("Starting classification training...")

# 1. Initialize a simple Classifier
# We use 'class_weight' because most flights are NOT delayed
rf_classifier = RandomForestClassifier(
    n_estimators=100,        # Start simple
    max_depth=10,            # A reasonable depth to prevent overfitting
    min_samples_leaf=50,     # Don't let it learn from tiny groups
    n_jobs=-1,               # Use all cores
    random_state=42,
    class_weight='balanced'  # <-- CRITICAL!
)

# 2. Train the model
start_time = time.time()
rf_classifier.fit(X_train, y_train)
print(f"Training finished in {time.time() - start_time:.2f} seconds.")

# 3. Get predictions for the Test set
y_pred = rf_classifier.predict(X_test)
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1] # Get 'Delayed' probability

# 4. Print new Classification Metrics
print("\n--- Test Set Metrics ---")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.3f}")
print(f"Recall:    {recall_score(y_test, y_pred):.3f}")     # <-- How many delays did we catch?
print(f"Precision: {precision_score(y_test, y_pred):.3f}")  # <-- How many of our alarms were real?
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.3f}") # <-- The best overall metric

### Hyperparameter Tuning

In [ ]:
def objective(trial):
    # 1. Define the parameters to search
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500), # Shortened for speed
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 100),
    }

    # 2. Create the classifier
    model = RandomForestClassifier(
        **params,
        n_jobs=-1,
        random_state=42,
        class_weight='balanced' # <-- Still critical
    )

    # 3. Train on a smaller sample to make tuning fast
    X_train_sample = X_train.sample(n=100000, random_state=42)
    y_train_sample = y_train.loc[X_train_sample.index]

    model.fit(X_train_sample, y_train_sample)

    # 4. Score it on the validation set
    # We want to maximize the ROC-AUC score
    val_preds_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_preds_proba)

    return auc

# --- Run the study ---
print("Starting Optuna study...")
# We want to MAXIMIZE the score
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20) # Run for 20 trials

print("\nBest trial:")
print(f"  Value (ROC-AUC): {study.best_value:.4f}")
print("  Params: ")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

# You can now use these best_params in your final model
best_params = study.best_params

### Model Training & Results

In [ ]:
# --- FINAL: Train Tuned Model, Print Metrics, and Plot ---

# 1. Initialize the FINAL Tuned Classifier
print("Using best params from Optuna study...")
rf_classifier = RandomForestClassifier(
    n_estimators=442,        # <-- From Optuna
    max_depth=8,             # <-- From Optuna
    min_samples_leaf=34,     # <-- From Optuna
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)

# 2. Train the model
print("Training the final model...")
start_time = time.time()
rf_classifier.fit(X_train, y_train)
print(f"Training finished in {time.time() - start_time:.2f} seconds.")

# 3. Get predictions for the Test set
print("Generating predictions...")
y_pred = rf_classifier.predict(X_test)
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

# 4. Print new Classification Metrics
print("\n--- Test Set Metrics ---")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.3f}")
print(f"Recall:    {recall_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred):.3f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.3f}")

# --- Plotting Results ---
print("\n--- Plotting Results ---")
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# 5. Plot Confusion Matrix
ConfusionMatrixDisplay.from_estimator(
    rf_classifier,
    X_test,
    y_test,
    ax=ax[0],
    cmap='Blues',
    normalize='true'
)
ax[0].set_title('Confusion Matrix (Normalized)')

# 6. Plot ROC Curve
RocCurveDisplay.from_estimator(
    rf_classifier,
    X_test,
    y_test,
    ax=ax[1]
)
ax[1].set_title('ROC-AUC Curve')
plt.show()

## General Model (awaiting)